# Le modèle de ségrégation de Schelling

En Décembre 2016 disparaissait Thomas C. Schelling. Economiste, récipiendaire du Prix de la Banque de Suède en sciences économiques en mémoire d’Alfred Nobel (communément appelé –et considéré– comme le prix Nobel d’Economie), il a travaillé sur de nombreux sujets, en particulier l’analyse des conflits.

Une de ses contributions a permis de mieux comprendre les phénomènes de ségrégation. Plus précisément, le propos de Schelling fut d’étu- dier la dynamique par laquelle des phénomènes de ségrégation extrêmes peuvent survenir, en dépit de préférences qui peuvent sembler faiblement discriminantes individuellement. Ainsi, même si chaque individu se dé- clare prêt à accepter une certaine proportion d’invidus « différents » dans son voisinage, le résultat final peut être que la population se regroupe en régions très homogènes.

Cette étude peut être menée à l’aide de modèles connus sous le nom d’automates cellulaires. Un des automates les plus célèbres est le jeu de la vie, proposé par John Conway en 1970. Il existe de nombreux autres automates cellulaires dont le comportement est relativement bien étudié, et la littérature est riche à ce sujet. Nous recommandons la lecture de l’article [2] de Jean-Paul Delahaye.

In [27]:
import numpy as np
import random
import copy

from matplotlib import pyplot as plt

# GLOBAL PARAMETERS FOR EXPERIMENTS
neighb = 4          # size of neighbourhood
threshold = 0.5     # threshold of satisfaction
maxIterations = 5   # max number of iteration for convergence
size = 22           # size of the line

Un état de l'automate cellulaire est représenté sous la forme d'une liste de 0 et de 1. Par exemple:

In [2]:
cells = [0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1]

La fonction ```str_state``` convertit un automate cellulaire en une chaîne de caractères pour l'afficher à l'écran.

In [4]:
def str_state(state):
    '''
    return the state as a string
    '''
    result = ""
    for i in state:
        result += str(i)
    return result

In [13]:
str_state(cells)

'0100011010011100111101'

La fonction ```hogeneinity_level``` calcule pour l'individu à la position c, le ratio d'individus du même type.

In [6]:
def homogeneinity_level(position, state):
    '''
    for a given individual at position and state
    returns the ratio of individuals of same type in neighbourhood
    '''
    my_color = state[position]
    count = 0
    nb_neighb = 0
    for i in range(1, neighb+1):
        if position+i < len(state):
            nb_neighb += 1
            if state[position+i] == my_color:
                count += 1
        if position-i >= 0:
            nb_neighb += 1
            if state[position-i] == my_color:
                count += 1
    return float(count / nb_neighb)

In [7]:
homogeneinity_level(1,cells)

0.2

In [8]:
def is_happy(position, state, verbose = False):
    '''
    returns whether individual at position is satisfied in a given state
    '''
    h = homogeneinity_level(position, state)
    if verbose:
        print(h)
    return h >= threshold

In [9]:
is_happy(1, cells)

False

Il est également possible d'afficher sous forme de chaine de caractère quels individus sont insatisfaits (notés 'X'). 

In [10]:
def str_unhappy(string):
    '''
    returns the string marking unhappy individuals with a 'X'
    '''
    result = ""
    for i in range(len(string)):
        if is_happy(i, string):
            result += " "
        else:
            result += "X"
    return result

In [15]:
print(str_state(cells))
print(str_unhappy(str_state(cells)))

0100011010011100111101
 X   XX  XXX  XX    X 


La fonction ```move_to``` déplace un individu vers une nouvelle position à droite ou à gauche. La priorité est donnée au déplacement à droite.

In [20]:
def move_to(c, p, s):
    '''
    c position initiale
    p position où l'on déplace l'agent
    moves individual c to position p, shifting other individuals
    s state
    and returns the resulting list
    '''
    new_s = copy.copy(s) # new list for result
    my_color = new_s[c]
    # gives priority to the right move in case of ties
    # I didn't find this specification in Schelling's paper
    if p > c: # moving to the right
        for i in range(c, p):
            new_s[i] = s[i+1]
            new_s[i+1] = my_color
    else:   # moving to the left

        for i in range(p, c):
            new_s[i+1] = s[i]
            new_s[p] = my_color
    return new_s

In [21]:
new_cells = move_to(1, 7, cells)
print("Avant le déplacement:", str_state(cells))
print("Après le déplacement:", str_state(new_cells))

Avant le déplacement: 0100011010011100111101
Après le déplacement: 0000110110011100111101


In [28]:
def move_to_nearest_satisfying(c, s, verbose=False):
    '''
    will move individual c to nearest satisfying location
    simulate the move and check whether satisfying
    note: very inefficient but simple solution
    '''

    move_limit = max(size-c, c)
    move_distance = 0
    new_s = []
    satisfied = False
    while move_distance < move_limit and not(satisfied):
        move_distance += 1
        new_s = copy.copy(s) # used to simulate the move
        if c+move_distance < size:
            new_s = move_to(c, c+move_distance, s)
            if is_happy(c+move_distance, new_s):
                satisfied = True
                if verbose:
                    print (c, "moved to:", c+move_distance)
        else: # trying to move left
            if c-move_distance >= 0:
                new_s = copy.copy(s)
                new_s = move_to(c, c-move_distance, s)
                satisfied = is_happy(c-move_distance, new_s)
                if verbose and satisfied:
                    print (c, " moved to:", c-move_distance)
    return new_s, satisfied

In [36]:
print ("Avant le déplacement:", str_state(cells))
new_cells, sat = move_to_nearest_satisfying(1, cells,True)
print ("Après le déplacement:", str_state(new_cells))

Avant le déplacement: 0100011010011100111101
1 moved to: 7
Après le déplacement: 0000110110011100111101


### Dynamique

La dynamique consiste ensuite à répéter les déplacements. Schelling suggère de considérer les individus un par un, en partant de la gauche, et de les faire se déplacer s'ils le peuvent. Un *tour* est terminé lorsque tous les individus ont été considérés, et on peut répéter ainsi les tours. Mais quand s'arrêter avec cette dynamique? 

On pourrait penser différents critères de convergence dans notre cas:
* lorsque tous les individus sont satisfaits. Malheureusement, rien ne garantit que le système puisse parvenir à un état où tous les individus sont satisfaits (cela peut arriver mais c'est plutôt exceptionnel). 
* lorsque plus aucun individu ne peut se déplacer. Ce critère est plus pertinent, mais il cache une difficulté: le système ne parvient pas non plus nécessairement dans un état où plus aucun individus ne peut se déplacer. Ce critère seul peut donc mener à des boucles infinies. 
* nous emploierons donc un garde-fou, qui consistera à poser un nombre maximal d'itérations. Si le système n'est pas stabilisé selon le critère précédent après ce nombre d'itérations, la dynamique s'arrête. 

In [50]:
###############################################################################
# GLOBAL DYNAMICS
###############################################################################
# Note: departs a little bit from Schelling's specification here
# Interesting problem of non convergence here when no maxIterations condition
# the penultimate individual moves to the last position, and so on


def dynamics(state, verbose = False, stepwise = False):
    '''
    departs a little bit from Schelling's specification here
    '''
    one_has_moved = True
    iterations = 0
    while one_has_moved and iterations < maxIterations:
        one_has_moved = False
        for i in range(len(state)):
            if not (is_happy(i, state)): # i wants to move
                state, moved = move_to_nearest_satisfying(i, state, False)
                one_has_moved = moved or one_has_moved
        if verbose:
            print ("Tour :", iterations)
            print(str_state(state))
            print(str_unhappy(state))
            #print(count_unhappy(state))
        if stepwise:
            input("Press Enter to continue...")
        iterations += 1
    return state

In [51]:
new_cells = dynamics(cells,True)

Tour : 0
0000100101010011111111
    X  X X  XX        
Tour : 1
0000000000111111111111
                      
Tour : 2
0000000000111111111111
                      


Pour cet état initial on a donc une convergence directe. La dynamique s'arrête au tour 2 car plus aucun individu ne souhaite se déplacer. Mais considérons un autre état initial:

In [47]:
cells2 = [1,1,1,1,0,0,0]
new_cells2 = dynamics (cells2,True)

Tour : 0
1111000
    XX 
Tour : 1
1111000
    XX 
Tour : 2
1111000
    XX 
Tour : 3
1111000
    XX 
Tour : 4
1111000
    XX 


Cette fois la dynamique s'arrête grâce à la limite sur le nombre d'itérations. Comprenez-vous pourquoi? 